Testing data positive feedback dynamic (20%)

In [21]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math
from IPython.display import clear_output

In [23]:
for tw in range(10):
    #clear_output()
    print(tw)

0
1
2
3
4
5
6
7
8
9


In [2]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
user_following = np.load('../Data/npy/user_following_1489.npy')[:100]
all_3374 = np.load('../Data/npy/all_3374D.npy')
user_category = np.load('../Data/npy/user_category_1489.npy')[:100]
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')[:100]

In [4]:
#The number of followers for each YouTuber
YouTuber_followers = np.sum(user_following, axis=0)
print(YouTuber_followers)
over5 = 0
for num in YouTuber_followers:
    if num >= 5:
        over5+=1
print('The num of followers over 5:',over5)

[ 8  0  2 24  1  0  0 11 47  7 25 32 10  1 16  1  6 24  3  2  1  6  4  0
 39 39 10 16 27 22  0  2 17 15  0  2  0  8  2  1 28 16  3 33  9 24  0  3
  0  9  0  0  0  2 28  2  2 18  0  3 23  0  2 39  0  1  1 65  0  2 18 21
  7  1 47 14 19  1  3 66  3  0 28  3 20  0  6  0]
The num of followers over 5: 42


In [5]:
print('user_following shape ',user_following.shape)
print('all_3374 shape ',all_3374.shape)
print('user_category shape ',user_category.shape)
print('YouTuber_category shape ',YouTuber_category.shape)

user_following shape  (100, 88)
all_3374 shape  (88, 3374)
user_category shape  (100, 17)
YouTuber_category shape  (88, 17)


In [6]:
user_category_norm = np.zeros(user_category.shape)
for i in range(len(user_category)):
    user_category_norm[i] = user_category[i]/np.max(user_category[i])
print('user_category after normalized by max...')
print('user_category_norm shape ',user_category_norm.shape)

user_category after normalized by max...
user_category_norm shape  (100, 17)


In [7]:
following_true = [0]*len(user_following)
for i in range(len(user_following)):
    each_user = []
    for j in range(len(user_following[i])):
        if user_following[i][j] == 1:
            each_user.append(j)
    following_true[i] = each_user
#print(following_true)

In [8]:
#最少跟最多的following 
minlen = 10000
maxlen = 0
num_of_follower = []
for i in range(len(following_true)):
    if len(following_true[i]) < minlen:
        minlen = len(following_true[i])
    if len(following_true[i]) > maxlen:
        maxlen = len(following_true[i])
    num_of_follower.append(len(following_true[i]))
print('Min number of followings ',minlen)
print('Max number of followings ',maxlen)

Min number of followings  5
Max number of followings  21


In [9]:
over_10 = 0
over_8 = 0
over_12 = 0
user_idx_over10 = []
avg_over10 = 0
for i in range(len(num_of_follower)):
    num = num_of_follower[i]
    if num >= 10:
        over_10 += 1
        user_idx_over10.append(i)
        avg_over10+=num
    if num >= 8:
        over_8 += 1
    if num >= 12:
        over_12 += 1
print('over 8:',over_8)
print('over 10:',over_10)
print('over 12:',over_12)
print('avg 10:',avg_over10/len(user_idx_over10))

over 8: 68
over 10: 56
over 12: 28
avg 10: 12.517857142857142


In [10]:
len(user_idx_over10)

56

# Training data and Testing data

In [11]:
test_amount = 10
yt_test_amount = 20

In [12]:
user_idx = [i for i in range(len(user_following))]
#user_idx = user_idx_over10
#test_idx is the number of user for testing
random.seed(3)
test_idx = random.sample(user_idx,test_amount)

In [13]:
# Training  and Testing
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        # random choose 2 true and 8 false for test 
        if len(temp_t)*0.2 < 1:
            t_for_test = random.sample(temp_t,1)
            f_for_test  = random.sample(temp_f,yt_test_amount-1)
        else:
            if len(temp_t)*0.2 > 5:
                t_num = int(round(float(len(temp_t)*0.2)))
            else:      
                t_num = 5
            #t_num = int(round(float(len(temp_t)*0.2)))
            t_for_test = random.sample(temp_t,t_num)
            f_for_test  = random.sample(temp_f,yt_test_amount-t_num)
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train

In [14]:
# Training  and Testing --New
train_t = [0]*(len(user_following))
train_f = [0]*(len(user_following))
# Testing 
test_t = [0]*test_amount
test_f = [0]*test_amount
test_pos = -1

for i in range(len(user_following)):
    t_for_train = []
    f_for_train = []
    if i not in test_idx: #if not in test id, just append it to true or false list
        for j in range(88):
            if user_following[i][j] == 1:
                t_for_train.append(j)
            else:
                f_for_train.append(j)
        train_t[i] = t_for_train
        train_f[i] = f_for_train
        
    else: #if in test id, choose 2 true and other 
        test_pos += 1
        temp_t = []
        temp_f = []
        for j in range(88):
            if user_following[i][j] == 1:
                temp_t.append(j)
            else:
                temp_f.append(j)
        print(len(temp_t),math.ceil(0.5*len(temp_t)))
        t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
        f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))
        
        test_t[test_pos] = t_for_test
        test_f[test_pos] = f_for_test
        
        #other for training
        t_for_train = [item for item in temp_t if not item in t_for_test]
        print(len(t_for_train ))
        f_for_train = [item for item in temp_f if not item in f_for_test]
        train_t[i] = t_for_train
        train_f[i] = f_for_train

5 3
2
6 3
3
14 7
7
11 6
5
5 3
2
6 3
3
7 4
3
11 6
5
10 5
5
9 5
4


In [15]:
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/test_amount
avg

4.5

In [16]:
# train_t[i] 代表的是user i positive feedback
print('The length of train_t:',len(train_t))
print('The length of train_f:',len(train_t))
print('The length of test_t:',len(test_t))
print('The length of test_f:',len(test_f))

The length of train_t: 100
The length of train_f: 100
The length of test_t: 10
The length of test_f: 10


# Recommendation  Module

In [17]:
n = len(user_following)
m = 88  
k = 128
l = 3374
embedding_dims = 150

In [34]:
def training(): 
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    loss_acc_list = []
    t0=time.time()

    #use_true=init_list_of_objects(136)
    #use_test=init_list_of_objects(136)

    #train_pair_t=[] #positive feedback
    #train_pair_f=[] #negative feedback
    train_yes_id=[] 
    for q in range(10):
        clear_output()
        print('Iteraction:',q)
        train_auc=0
        total_loss=0
        xuij_auc=0
        length = 0
        for z in range(n):
            """
            yes 用來存放選擇到的YouTuber feature (for auxilary)
            yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
            r_3 用來存放user 對該YouTuber種類的偏好(取max)
            """
            yes=[]
            yesr=[]
        
        
            sample=random.sample(train_t[z],len(train_t[z])) #選全部的Positive
            train_yes_id.append(sample) #sample全部丟進去
        
            #sample=random.sample(train_t[z]+train_f[z],len(train_t[z])+len(train_f[z]))
        
            #change
            r_3=np.zeros(len(sample)) 
         
            for b in range(len(sample)):
                yes.append(all_3374[sample[b]])
                yesr.append(YouTuber_category[sample[b]]*user_category_norm[z])
                #print('YouTuber_category ', YouTuber_category[sample[k]])
                #print('User_category ',user_category_norm[z])
            #print(yesr)
        
            for b in range(len(yesr)):
                r_3[b]=max(yesr[b])
            #print('r_3:',r_3)
        
            yes=np.array(yes)
        
            #not_used_list = list(set(train_t[z]).difference(set(sample)))
        
            #取positive 
            train_t_sample = random.sample(train_t[z],len(train_t[z]))
            #print('number of positive feedback', len(train_t_sample))
        
            train_f_sample = random.sample(train_f[z],20)
            for ta in train_t_sample:
                #print(ta,'--> positive feedback')
            
                pos = sample.index(ta)
                #new_sample = np.delete(sample,[pos])
                #new_yes = np.delete(yes,[pos],axis=0)
                #new_r_3 = np.delete(r_3,[pos])
                new_sample = sample
                new_yes = yes
                new_r_3 = r_3
                #print(len(yes),len(new_yes))
                #print(yes)
                #print(new_yes)
            
            
            
                #ta=random.choice(train_t[z]) #ta is true positve photo
                #train_pair_t.append(ta)
                image_1=np.expand_dims(all_3374[ta],0) #(1,2048)
                #print('Image_1 shape ',image_1.shape)
                #train_f_sample = random.sample(train_f[z],20)
                #print('True:',train_t_sample,'Now:',ta)
                #print('False:',train_f_sample)
                for b in train_f_sample:
                    #print('likes:',ta,';Not likes:',b)
                    #b=random.choice(train_f[z])  #b is no feedback photo
                    #train_pair_f.append(b)
                    image_2=np.expand_dims(all_3374[b],0) #(1,2048)
                    #print('Image_2 shape',image_2.shape)
            
                    #use_test[z].append(b)
                    _w1,_wu,_wy,_wa,_wv,_beta,_norm_par,_a_list,r3,_auc, _loss,_=sess.run([w1,wu,wy,wa,wv,beta,norm_par,a_list,a_list_soft,auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b], xf: new_yes , l_id:new_sample, l_id_len:[len(new_sample)],r:new_r_3,
                                        image_i:image_1,image_j:image_2})
                
                    #print(XUIJ)
                    #print('loss=',_loss)
                    #print('auc=',_auc)
                    #print('user positive negative',z,ta,b)
                    print('u,vi,vj',_norm_par[:3])
                    print('w1,wu,wy,wa,wv',_norm_par[3:8])
                    print('beta',_norm_par[8:9])
                    print('Embedding',_norm_par[-1])
                
                    print('alpha softmax:',r3)
                    print('before softmax:',_a_list)
                    print('---------------------------------------------------')
        
                    train_auc+=_auc
                    total_loss+=_loss
                    length += 1
                #now1+=1
        
            #np.savez('../Data/latent_factor/YRM_up10/'+str(q)+'_'+str(z)+'.npz', User=U_history, YouTuber=Y_history)
    
        #print('mine:',xuij_auc/136)    
        #print('a_list_soft:',r3)
        #print("total_loss:-----------------", total_loss/length)
        #print("train_auc:-------------------", train_auc/length)
        #print('---------------------------------------------------------------------------------------------------')
        loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
        #print('time:',time.time()-t0,' sec')
    #print('Total cost ',time.time()-t0,' sec')   
    U, Y, A, E, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, W1, Wu, Wy, Wa, Wv,Beta])
    return U, Y, A, E, A1, Au, Ay, Aa, Av,B

In [ ]:
par_weights = [0.01,0.001,0.0001,0.00001]
beta_weights = [1,0.1,0.01]
Embedding_weights = [1,0.1,0.01]
for paru_weight in par_weights:
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                """
                n: the number of users
                m: the number of YouTubers
                k: latent dims
                l: feature dims
                """
                tf.reset_default_graph()

                user = tf.placeholder(tf.int32,shape=(1,))
                i = tf.placeholder(tf.int32, shape=(1,))
                j = tf.placeholder(tf.int32, shape=(1,))

                #多少個auxliary 
                xf = tf.placeholder(tf.float32, shape=(None,l))
                l_id = tf.placeholder(tf.int32, shape=(None,))
                l_id_len = tf.placeholder(tf.int32,shape=(1,))
                r = tf.placeholder(tf.float32,shape=(None,))


                image_i = tf.placeholder(tf.float32, shape=(1,l))
                image_j = tf.placeholder(tf.float32, shape=(1,l))

                with tf.variable_scope("item_level"):
                    user_latent = tf.get_variable("user_latent", [n, k],
                                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
                    item_latent = tf.get_variable("item_latent", [m, k],
                                                      initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
                    aux_item = tf.get_variable("aux_item", [m, k],
                                                      initializer=tf.random_normal_initializer(0,0.1,seed=3))
                    W1 = tf.get_variable("W1", [n, k],
                                                      initializer=tf.contrib.layers.xavier_initializer())
                    Wu = tf.get_variable("Wu", [n,k,k],
                                                      initializer=tf.contrib.layers.xavier_initializer())
                    Wy = tf.get_variable("Wy", [m,k,k],
                                                     initializer=tf.contrib.layers.xavier_initializer())
                    Wa = tf.get_variable("Wa", [m,k,k],  ###################
                                                     initializer=tf.contrib.layers.xavier_initializer())
                    Wv = tf.get_variable("Wv", [m,k,l],  ###################
                                                    initializer=tf.contrib.layers.xavier_initializer())



                    aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
                    ########## Error part, how to get auxisize dynamically
                    ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])

                with tf.variable_scope('feature_level'):
                    embedding = tf.get_variable("embedding", [embedding_dims,l],
                                                     initializer=tf.contrib.layers.xavier_initializer())
                    Beta = tf.get_variable("beta", [n,embedding_dims],
                                             # initializer=tf.contrib.layers.xavier_initializer())
                                                     initializer=tf.random_normal_initializer(0.01,0.001,seed=10))

                #lookup the latent factors by user and id
                u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                vi = tf.nn.embedding_lookup(item_latent, i) 
                vj = tf.nn.embedding_lookup(item_latent, j)

                w1 = tf.nn.embedding_lookup(W1, user) #(1*k)
                #wu = tf.squeeze(Wu)
                wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(k*k)
                wy = tf.squeeze(tf.nn.embedding_lookup(Wy, i)) #(k*k)
                #wa = tf.squeeze(Wa)
                wa = tf.squeeze(tf.nn.embedding_lookup(Wa, i)) #(k*k)
                #wv = tf.squeeze(Wv)
                #wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(k,l)
                wv = tf.squeeze(tf.nn.embedding_lookup(Wv, i)) #(k,l)
                beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


                a_list=tf.Variable([])
                q = tf.constant(0)
                def att_cond(q,a_list):
                    return tf.less(q,l_id_len[0])
                def att_body(q,a_list):
                    xfi = tf.expand_dims(xf[q],0) #(1,l)

                    a_list = tf.concat([a_list,[(tf.matmul( w1, tf.nn.relu( tf.matmul(wu, u, transpose_b=True) +
                            tf.matmul(wy, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                            tf.matmul(wa, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                            tf.matmul(wv, xfi, transpose_b=True)))[0][0])*r[q]]],0)
                    q += 1
                    return q,  a_list

                _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                a_list_soft=tf.nn.softmax(a_list)


                aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
                q = tf.constant(0)
                def sum_att_cond(q,aux_np):
                    return tf.less(q,l_id_len[0])

                def sum_att_body(q,aux_np):
                    #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                    q += 1
                    return q, aux_np

                _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                """
                for q in range(3): #取q個auxliary item
                    aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                """

                aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                #tf.print('aux attention:',aux_np)
                aux_np+=u #user_latent factor + sum (alpha*auxilary)
                aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
                feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
                feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
                only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
                only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

                #矩陣中對應函數各自相乘
                # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

                xuij = tf.subtract(xui,xuj)

                norm_par = [tf.reduce_sum(tf.multiply(u, u)),tf.reduce_sum(tf.multiply(vi, vi)),tf.reduce_sum(tf.multiply(vj, vj)),
                           tf.reduce_sum(tf.multiply(w1, w1)),tf.reduce_sum(tf.multiply(wu, wu)),tf.reduce_sum(tf.multiply(wy, wy)),
                           tf.reduce_sum(tf.multiply(wa, wa)),tf.reduce_sum(tf.multiply(wv,wv)),tf.reduce_sum(tf.multiply(beta,beta)),tf.reduce_sum(tf.multiply(embedding,embedding))]
                l2_norm = tf.add_n([
                            0.0001 * tf.reduce_sum(tf.multiply(u, u)),
                            0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
                            0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),

                            paru_weight * tf.reduce_sum(tf.multiply(w1, w1)),
                            paru_weight * tf.reduce_sum(tf.multiply(wu, wu)),
                            pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                            pary_weight * tf.reduce_sum(tf.multiply(wa, wa)),
                            pary_weight * tf.reduce_sum(tf.multiply(wv,wv)),

                            beta_weight * tf.reduce_sum(tf.multiply(beta,beta)),
                            Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),

                          ])

                loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
                auc = tf.reduce_mean(tf.to_float(xuij > 0))
                
                Ur, Yr, Ar, Er, A1r, Aur, Ayr, Aar, Avr,Br = training()
                np.savez('../Data/grid_search_weight/0106/'+str(pary_weight)+'_'+str(pary_weight)+'_'+str(beta_weight)+'_'+str(Embedding_weight)+'.npz', 
                         U=Ur, Y=Yr, A=Ar, E=Er, W1=A1r, Wu=Aur, Wy=Ayr, Wa=Aar, Wv=Avr,B=Br)
                print('Finish,',str(paru_weight)+'_'+str(pary_weight)+'_'+str(beta_weight)+'_'+str(Embedding_weight))


Iteraction: 0
u,vi,vj [1.2751031, 1.288435, 1.4425361]
w1,wu,wy,wa,wv [1.0733064, 1.2770157, 1.4520361, 1.4539424, 2.8059926]
beta [0.0152277835]
Embedding 287.11282
alpha softmax: [0.14437616 0.14669777 0.14235668 0.14283767 0.14001232 0.14263505
 0.14108433]
before softmax: [0.0424832  0.05843559 0.02839684 0.03176989 0.01179155 0.03035037
 0.01941888]
---------------------------------------------------
u,vi,vj [1.2731178, 1.2885152, 1.4579873]
w1,wu,wy,wa,wv [1.0713034, 1.2522073, 1.4255025, 1.427408, 2.6208057]
beta [0.015229]
Embedding 286.2566
alpha softmax: [0.14433324 0.14676356 0.14238587 0.14277665 0.14005871 0.14272691
 0.14095509]
before softmax: [0.04055868 0.05725667 0.02697461 0.02971545 0.01049558 0.02936698
 0.01687513]
---------------------------------------------------
u,vi,vj [1.271124, 1.288598, 1.3661444]
w1,wu,wy,wa,wv [1.0693028, 1.2277169, 1.3992873, 1.4011917, 2.443817]
beta [0.015224281]
Embedding 285.3341
alpha softmax: [0.14428936 0.1467702  0.14240348 0.14

u,vi,vj [1.2121704, 1.3051841, 1.4146029]
w1,wu,wy,wa,wv [1.0106614, 0.6488841, 1.1576871, 1.1604555, 1.1230985]
beta [0.018764708]
Embedding 255.5739
alpha softmax: [0.14170744 0.14481819 0.14298774 0.14287363 0.14305544 0.14211214
 0.14244539]
before softmax: [0.01308472 0.03479914 0.02207898 0.02128065 0.02255233 0.01593659
 0.01827873]
---------------------------------------------------
u,vi,vj [1.2102506, 1.305743, 1.6286448]
w1,wu,wy,wa,wv [1.0087562, 0.63448817, 1.130393, 1.1331494, 1.0094626]
beta [0.018944403]
Embedding 254.53062
alpha softmax: [0.14170584 0.14477913 0.14297202 0.1428394  0.14311714 0.14217478
 0.14241163]
before softmax: [0.01236035 0.03381633 0.02125605 0.02032802 0.0222705  0.0156641
 0.01732875]
---------------------------------------------------
u,vi,vj [1.2083342, 1.3063041, 1.4059627]
w1,wu,wy,wa,wv [1.0068543, 0.62037325, 1.1033905, 1.1061338, 0.90416867]
beta [0.01912718]
Embedding 253.4878
alpha softmax: [0.1417312  0.14478469 0.1429539  0.14278913 0

w1,wu,wy,wa,wv [0.9495369, 0.300229, 1.3029866, 1.3185294, 1.835266]
beta [0.02266008]
Embedding 224.02669
alpha softmax: [0.14213747 0.14180489 0.14149973 0.14787374 0.14426157 0.14193487
 0.1404877 ]
before softmax: [-0.0135321  -0.01587457 -0.0180289   0.02603213  0.0013015  -0.01495845
 -0.02520673]
---------------------------------------------------
u,vi,vj [1.148728, 1.4132719, 1.4411042]
w1,wu,wy,wa,wv [0.94773996, 0.2930199, 1.2654483, 1.2807869, 1.6281726]
beta [0.022824459]
Embedding 223.13922
alpha softmax: [0.14202581 0.14199005 0.14179398 0.14743634 0.14417551 0.14200172
 0.14057657]
before softmax: [-0.01081304 -0.01106494 -0.01244685  0.02657455  0.0042095  -0.01098276
 -0.02106951]
---------------------------------------------------
u,vi,vj [1.1469187, 1.4139813, 0.9760159]
w1,wu,wy,wa,wv [0.9459461, 0.28596765, 1.2273053, 1.2424331, 1.4318316]
beta [0.022987954]
Embedding 222.24904
alpha softmax: [0.14189437 0.14216928 0.14205106 0.14699805 0.14407657 0.14211297
 0.140

u,vi,vj [1.090643, 1.136903, 1.1303911]
w1,wu,wy,wa,wv [0.89018303, 0.12711394, 0.74935496, 0.74728787, 0.113074556]
beta [0.028618578]
Embedding 194.29944
alpha softmax: [0.14234667 0.14368083 0.14401877 0.14277093 0.14293543 0.14274558
 0.14150175]
before softmax: [ 0.00128257  0.01061157  0.01296076  0.0042586   0.00541022  0.00408113
 -0.00467068]
---------------------------------------------------
u,vi,vj [1.088934, 1.1952114, 1.44015]
w1,wu,wy,wa,wv [0.8884903, 0.12380755, 1.4481994, 1.4591091, 2.801663]
beta [0.028768761]
Embedding 193.4508
alpha softmax: [0.143239   0.14084543 0.14500226 0.1443373  0.14209615 0.14001845
 0.14446144]
before softmax: [ 0.00248303 -0.0143686   0.01471766  0.01012122 -0.0055278  -0.0202575
  0.01098103]
---------------------------------------------------
u,vi,vj [1.0872277, 1.1953709, 1.4546319]
w1,wu,wy,wa,wv [0.88680065, 0.12057942, 1.4222736, 1.433014, 2.6201596]
beta [0.028916318]
Embedding 192.60332
alpha softmax: [0.1431349  0.14079282 0.1450

Embedding 169.65411
alpha softmax: [0.14117572 0.1441733  0.1414144  0.1441583  0.14500132 0.1416641
 0.14241283]
before softmax: [-0.03503887 -0.01402817 -0.03334964 -0.01413217 -0.00830144 -0.03158545
 -0.02631418]
---------------------------------------------------
u,vi,vj [1.0373621, 1.137558, 1.5015848]
w1,wu,wy,wa,wv [0.8374561, 0.0529251, 0.9650897, 0.9521507, 0.44696873]
beta [0.035057638]
Embedding 168.91705
alpha softmax: [0.1411994  0.14431949 0.14138758 0.14407632 0.14477803 0.14180757
 0.14243162]
before softmax: [-0.03179213 -0.00993561 -0.03046014 -0.01162191 -0.0067634  -0.02749405
 -0.02310313]
---------------------------------------------------
u,vi,vj [1.0357426, 1.1374161, 1.411479]
w1,wu,wy,wa,wv [0.8358553, 0.051437743, 0.9201709, 0.9077257, 0.34783965]
beta [0.035287056]
Embedding 168.1832
alpha softmax: [0.14116761 0.14442144 0.14137389 0.14392656 0.14460272 0.1420509
 0.1424569 ]
before softmax: [-0.02871644 -0.0059287  -0.0272564  -0.00936112 -0.00467437 -0.02

...

u,vi,vj [0.7099956, 1.3885412, 1.5678194]
w1,wu,wy,wa,wv [0.49876708, 0.00010052537, 4.1860934e-05, 4.2565614e-05, 0.004112886]
beta [0.049002778]
Embedding 7.524557
alpha softmax: [0.09096316 0.09097376 0.09081576 0.09094585 0.09080329 0.09091145
 0.09094191 0.09096792 0.09088557 0.09093195 0.09085942]
before softmax: [ 2.1708349e-04  3.3369137e-04 -1.4046505e-03  2.6838119e-05
 -1.5419923e-03 -3.5147375e-04 -1.6629008e-05  2.6936055e-04
 -6.3620292e-04 -1.2597970e-04 -9.2406309e-04]
---------------------------------------------------
u,vi,vj [0.70616996, 1.3886467, 1.3663459]
w1,wu,wy,wa,wv [0.49531162, 9.3532064e-05, 9.914909e-05, 0.00010256392, 0.009725525]
beta [0.04883979]
Embedding 7.5067744
alpha softmax: [0.09100364 0.09095503 0.09084656 0.09097318 0.09080335 0.0909083
 0.0909546  0.09093469 0.09094073 0.09082329 0.09085663]
before softmax: [ 5.21302922e-04 -1.29739055e-05 -1.20626413e-03  1.86470948e-04
 -1.68209593e-03 -5.27015014e-04 -1.77345737e-05 -2.36627093e-04
 -1.7026

u,vi,vj [0.65337616, 1.3938715, 1.4719558]
w1,wu,wy,wa,wv [0.44790524, 2.2473183e-05, 2.1816115e-05, 2.1821916e-05, 0.0020773213]
beta [0.047642864]
Embedding 7.2605267
alpha softmax: [0.09087615 0.09088143 0.09094466 0.09088629 0.09089281 0.09091748
 0.0910735  0.09083711 0.09090658 0.09087518 0.09090882]
before softmax: [-2.1893389e-03 -2.1311645e-03 -1.4357029e-03 -2.0777816e-03
 -2.0060216e-03 -1.7345083e-03 -1.9998732e-05 -2.6189608e-03
 -1.8545316e-03 -2.1999425e-03 -1.8298385e-03]
---------------------------------------------------
u,vi,vj [0.6501313, 1.5431437, 1.4203987]
w1,wu,wy,wa,wv [0.44500926, 1.9937534e-05, 0.00035840995, 0.00036981504, 0.037359864]
beta [0.047501612]
Embedding 7.2443066
alpha softmax: [0.09125943 0.09116008 0.09075089 0.09087609 0.09087766 0.09046112
 0.09085253 0.09117841 0.09073793 0.09104288 0.090803  ]
before softmax: [ 4.4166171e-03  3.3273820e-03 -1.1713926e-03  2.0724323e-04
  2.2445372e-04 -4.3694791e-03 -5.2090010e-05  3.5283573e-03
 -1.3143218

u,vi,vj [0.6048899, 1.5478075, 1.4714493]
w1,wu,wy,wa,wv [0.40486705, 2.9376768e-06, 4.00851e-05, 4.3095723e-05, 0.004122857]
beta [0.049272925]
Embedding 7.2568884
alpha softmax: [0.09097166 0.09096932 0.09086662 0.09088652 0.09090651 0.09084666
 0.09085041 0.09096383 0.09089422 0.09093673 0.09090745]
before softmax: [ 1.3272103e-03  1.3014546e-03  1.7185287e-04  3.9081243e-04
  6.1071600e-04 -4.7854570e-05 -6.6375860e-06  1.2410800e-03
  4.7561390e-04  9.4303186e-04  6.2103476e-04]
---------------------------------------------------
u,vi,vj [0.6020824, 1.5482109, 1.2717696]
w1,wu,wy,wa,wv [0.40239108, 2.6191988e-06, 3.3145785e-05, 3.475559e-05, 0.0034345163]
beta [0.049550097]
Embedding 7.2683196
alpha softmax: [0.09095833 0.09095997 0.09086944 0.09090476 0.09089785 0.09086412
 0.09083672 0.09094258 0.09090608 0.09093343 0.09092675]
before softmax: [ 1.3346694e-03  1.3526676e-03  3.5690388e-04  7.4554159e-04
  6.6955358e-04  2.9839957e-04 -3.1719010e-06  1.1615172e-03
  7.6008547e-04

u,vi,vj [0.5275401, 1.0216994, 1.5401783]
w1,wu,wy,wa,wv [0.3373813, 1.5095573e-07, 5.4285476e-05, 6.237198e-05, 0.0056850365]
beta [0.05005289]
Embedding 7.224389
alpha softmax: [0.09093961 0.09094486 0.09084018 0.09088788 0.0908751  0.09084629
 0.09093719 0.09097809 0.09091162 0.09093998 0.09089918]
before softmax: [ 1.7234346e-05  7.4928001e-05 -1.0767407e-03 -5.5176712e-04
 -6.9242012e-04 -1.0095543e-03 -9.4360403e-06  4.4028045e-04
 -2.9059296e-04  2.1219395e-05 -4.2745043e-04]
---------------------------------------------------
u,vi,vj [0.52533454, 1.0218232, 1.4572353]
w1,wu,wy,wa,wv [0.3354811, 1.342734e-07, 4.3274707e-05, 4.6458543e-05, 0.0043420973]
beta [0.050180554]
Embedding 7.2152853
alpha softmax: [0.09094451 0.09093303 0.09088203 0.09090657 0.09087393 0.09086274
 0.09094116 0.09093857 0.09090119 0.09091884 0.09089739]
before softmax: [ 3.1155505e-05 -9.5065974e-05 -6.5609661e-04 -3.8607646e-04
 -7.4529846e-04 -8.6831668e-04 -5.6238141e-06 -3.4140234e-05
 -4.4532231e-04 

u,vi,vj [0.46750188, 1.7212248, 1.2702003]
w1,wu,wy,wa,wv [0.28620484, 5.753627e-09, 0.00020126285, 0.00020897973, 0.01359509]
beta [0.054393683]
Embedding 7.169071
alpha softmax: [0.09080646 0.09085659 0.09083685 0.09088536 0.09097915 0.0909744
 0.09122044 0.09081373 0.09084623 0.09087491 0.0909059 ]
before softmax: [-4.5841411e-03 -4.0323697e-03 -4.2497213e-03 -3.7156628e-03
 -2.6842447e-03 -2.7365084e-03 -3.5701931e-05 -4.5042257e-03
 -4.1463939e-03 -3.8306308e-03 -3.4896464e-03]
---------------------------------------------------
u,vi,vj [0.46569592, 1.7207581, 1.3522418]
w1,wu,wy,wa,wv [0.28468445, 5.254443e-09, 0.00025706342, 0.00026803178, 0.015129342]
beta [0.054072782]
Embedding 7.1785913
alpha softmax: [0.0908205  0.09082215 0.09088666 0.09091859 0.09097248 0.09101799
 0.09121436 0.09076776 0.0908602  0.0908314  0.0908879 ]
before softmax: [-4.3564513e-03 -4.3383320e-03 -3.6282360e-03 -3.2769116e-03
 -2.6845208e-03 -2.1842741e-03 -2.9143952e-05 -4.9373908e-03
 -3.9194897e-03 

In [ ]:
def testing(U, Y, A, E, A1, Au, Ay, Aa, Av,B):
    #print(A1)
    result=np.zeros((test_amount,88))
    RS=np.zeros((test_amount,88))
    #test_idx --> Test 的 index
    max1 = 0
    maxu = 0
    maxy = 0
    maxa = 0
    maxv = 0
    min1 = 100000000000000000
    minu = 100000000000000000
    miny = 100000000000000000
    mina = 100000000000000000
    minv = 100000000000000000
    test_yes_id=[]
    for s in range(test_amount):
        print(s,test_idx[s])

        yes=[]
        sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        #sample=result_yes_id[now]
        test_yes_id.append(sample)
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
            #print(test_idx[s])
            #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))

            #Observe each part in attention , below par are all (128,1)
            testW1 = np.sum(A1[test_idx[s]])
            #print(A1[test_idx[s]])
            WuUu = np.sum(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T))
            WyYy = np.sum(np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T))
            WaAa = np.sum(np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T))
            WvVy = np.sum(np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))
            print('The sum of each par -->\nw1:',testW1,'\nWuU:',WuUu,'\nwyY:',WyYy,'\nWaA:',WaAa,'\nWvV:',WvVy)
            if testW1 > max1:
                max1 = testW1
            if testW1 < min1:
                min1 = testW1
            if WuUu > maxu:
                maxu = WuUu
            if WuUu < minu:
                minu = WuUu
            if WyYy > maxy:
                maxy = WyYy
            if WyYy < miny:
                miny = WyYy
            if WaAa > maxa:
                maxa = WaAa
            if WaAa < mina:
                mina = WaAa
            if WvVy > maxv:
                mxv = WvVy
            if WvVy < minv:
                minv = WvVy
            #Have w1
            #alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
            #                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))))*r
            #Without w1
            alpha[a]=np.sum((relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
                                np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))))*r
        mul=np.zeros((1,128))
        print('alpha------------',alpha)
        print('softmax alpha--------------',softmax(alpha))
        for i in range(len(sample)):
            mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
        for k in range(88):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
    #print(max1,maxu,maxy,maxa,maxv)
    #print(min1,minu,miny,mina,minv)
    return RS

In [90]:
import os 
#從grid_search_weight中找尋不同的file 
path = '../Data/grid_search_weight/0105/'
files = os.listdir(path)
#U, Y, A, E, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, W1, Wu, Wy, Wa, Wv,Beta])
for file in files:
    par_data = np.load(path+file)
    U = par_data['U']
    Y = par_data['Y']
    A = par_data['A']
    E = par_data['E']
    W1 = par_data['W1']
    Wu = par_data['Wu']
    Wy = par_data['Wy']
    Wa = par_data['Wa']
    Wv = par_data['Wv']
    B = par_data['B']
    RS = testing(U, Y, A, E, W1, Wu, Wy, Wa, Wv,B)

0 67
The sum of each par -->
w1: -6.354392e-11 
WuU: -2.6722195e-13 
wyY: -0.0026863154 
WaA: 4.682297e-07 
WvV: -1.831835920711231e-05
The sum of each par -->
w1: -6.354392e-11 
WuU: -2.6722195e-13 
wyY: 0.0014379378 
WaA: -9.4507277e-07 
WvV: -4.413202364227753e-05
The sum of each par -->
w1: -6.354392e-11 
WuU: -2.6722195e-13 
wyY: 0.0040578493 
WaA: -1.2154092e-06 
WvV: -1.4642087787816175e-05
The sum of each par -->
w1: -6.354392e-11 
WuU: -2.6722195e-13 
wyY: -0.00034374977 
WaA: -1.9116997e-06 
WvV: -4.1077026532084126e-05
The sum of each par -->
w1: -6.354392e-11 
WuU: -2.6722195e-13 
wyY: 0.00057212124 
WaA: 9.620932e-07 
WvV: -3.697121155308329e-05
The sum of each par -->
w1: -6.354392e-11 
WuU: -2.6722195e-13 
wyY: -0.0027414353 
WaA: -5.3686904e-07 
WvV: -2.696141722865594e-05
The sum of each par -->
w1: -6.354392e-11 
WuU: -2.6722195e-13 
wyY: 0.0020623913 
WaA: 1.4030488e-06 
WvV: 2.66667493662663e-06
The sum of each par -->
w1: -6.354392e-11 
WuU: -2.6722195e-13 
wyY: 0.

The sum of each par -->
w1: 1.5683126e-08 
WuU: 9.862695e-07 
wyY: 0.009147426 
WaA: 1.2031471e-05 
WvV: 0.00012417368405587351
The sum of each par -->
w1: 1.5683126e-08 
WuU: 9.862695e-07 
wyY: 0.0040578493 
WaA: 1.7360414e-06 
WvV: 0.00013438027161580266
alpha------------ [7.81521000e-05 1.04871867e-02 0.00000000e+00 7.63389153e-04
 9.29299989e-03 2.88577484e-02 1.31233902e-02]
softmax alpha-------------- [0.14158978 0.14307129 0.14157872 0.14168684 0.14290054 0.14572388
 0.14344895]
9 11
The sum of each par -->
w1: 1.2993729e-07 
WuU: 1.5224785e-07 
wyY: 0.0040578493 
WaA: -9.205244e-06 
WvV: -0.00018425255866934113
The sum of each par -->
w1: 1.2993729e-07 
WuU: 1.5224785e-07 
wyY: -0.0027414353 
WaA: 1.7336348e-05 
WvV: -0.0009802354955255103
The sum of each par -->
w1: 1.2993729e-07 
WuU: 1.5224785e-07 
wyY: 0.004048094 
WaA: 2.3759287e-05 
WvV: -0.0008186628180384438
The sum of each par -->
w1: 1.2993729e-07 
WuU: 1.5224785e-07 
wyY: 0.0042144028 
WaA: 5.4099204e-05 
WvV: -0.000

7 77
The sum of each par -->
w1: -7.5132903e-06 
WuU: 1.7817547e-07 
wyY: -0.00028575782 
WaA: 2.0171696e-05 
WvV: -0.0011813612198725157
The sum of each par -->
w1: -7.5132903e-06 
WuU: 1.7817547e-07 
wyY: -0.0015559997 
WaA: 4.6768942e-05 
WvV: -0.0003870117924973087
The sum of each par -->
w1: -7.5132903e-06 
WuU: 1.7817547e-07 
wyY: -0.00024206762 
WaA: -5.8972455e-05 
WvV: -0.0019120985287012743
The sum of each par -->
w1: -7.5132903e-06 
WuU: 1.7817547e-07 
wyY: -0.0011592356 
WaA: -9.7967066e-05 
WvV: -0.0016439780207427558
The sum of each par -->
w1: -7.5132903e-06 
WuU: 1.7817547e-07 
wyY: -0.0039356034 
WaA: 7.2202834e-05 
WvV: -0.0008829094693161995
alpha------------ [0.01694574 0.01239416 0.00981025 0.00917789 0.00203994]
softmax alpha-------------- [0.20137679 0.20046229 0.19994498 0.19981858 0.19839737]
8 17
The sum of each par -->
w1: 6.854208e-09 
WuU: 2.5878654e-07 
wyY: -0.00097459834 
WaA: 2.0797847e-06 
WvV: 0.00013568153364301665
The sum of each par -->
w1: 6.85420

5 24
The sum of each par -->
w1: -8.8797414e-11 
WuU: 2.2007578e-07 
wyY: 0.006847838 
WaA: -1.7492306e-05 
WvV: -0.0003412825013944424
The sum of each par -->
w1: -8.8797414e-11 
WuU: 2.2007578e-07 
wyY: 0.003996422 
WaA: 1.2072671e-05 
WvV: -0.0003797828150261558
The sum of each par -->
w1: -8.8797414e-11 
WuU: 2.2007578e-07 
wyY: 0.00017524976 
WaA: -8.5163465e-06 
WvV: -0.0003158943111943529
The sum of each par -->
w1: -8.8797414e-11 
WuU: 2.2007578e-07 
wyY: -0.0036470285 
WaA: 1.1200823e-05 
WvV: -0.00034240034185436933
The sum of each par -->
w1: -8.8797414e-11 
WuU: 2.2007578e-07 
wyY: -0.00086294406 
WaA: 1.7487451e-05 
WvV: -0.00032030903065105153
The sum of each par -->
w1: -8.8797414e-11 
WuU: 2.2007578e-07 
wyY: -0.0044637453 
WaA: 1.0303531e-05 
WvV: -0.0003612939642267614
The sum of each par -->
w1: -8.8797414e-11 
WuU: 2.2007578e-07 
wyY: -0.0042776405 
WaA: -2.2749646e-08 
WvV: -0.0002895663115713641
alpha------------ [0.05425196 0.02143989 0.0238348  0.01444216 0.0065

3 78
The sum of each par -->
w1: -4.777089e-06 
WuU: 3.9124616e-06 
wyY: -0.004091584 
WaA: 4.1268795e-05 
WvV: -0.0005122436341732162
The sum of each par -->
w1: -4.777089e-06 
WuU: 3.9124616e-06 
wyY: 0.0014591094 
WaA: 7.390692e-05 
WvV: -0.0003791075430473438
The sum of each par -->
w1: -4.777089e-06 
WuU: 3.9124616e-06 
wyY: 5.0490955e-05 
WaA: 7.070225e-05 
WvV: 7.041841745263338e-05
The sum of each par -->
w1: -4.777089e-06 
WuU: 3.9124616e-06 
wyY: -2.808229e-05 
WaA: -9.863652e-07 
WvV: 0.0004204100726984077
The sum of each par -->
w1: -4.777089e-06 
WuU: 3.9124616e-06 
wyY: 0.0010813709 
WaA: -8.5189226e-05 
WvV: -0.0002651031226911832
alpha------------ [0.02456808 0.01211237 0.04197239 0.00818551 0.00643641]
softmax alpha-------------- [0.20116835 0.1986782  0.2047002  0.19789955 0.1975537 ]
4 47
The sum of each par -->
w1: -3.226996e-06 
WuU: -3.8673925e-06 
wyY: -0.006210181 
WaA: -9.683598e-05 
WvV: 2.053089284703518e-05
The sum of each par -->
w1: -3.226996e-06 
WuU: -3.

...

The sum of each par -->
w1: -7.459696e-13 
WuU: 2.7342665e-13 
wyY: 7.011115e-11 
WaA: -2.4949287e-08 
WvV: 1.091343924933622e-05
The sum of each par -->
w1: -7.459696e-13 
WuU: 2.7342665e-13 
wyY: 1.7024096e-08 
WaA: 3.2278396e-07 
WvV: 1.4891959549963734e-05
alpha------------ [9.27234482e-06 6.49190854e-05 1.27478703e-05 5.68548097e-05
 4.64645042e-05 4.48237319e-02 3.59818899e-05 2.90867749e-06
 1.34691196e-05 2.06861240e-05]
softmax alpha-------------- [0.09954197 0.09954751 0.09954232 0.09954671 0.09954567 0.10410436
 0.09954463 0.09954134 0.09954239 0.09954311]
2 98
The sum of each par -->
w1: -1.2701587e-06 
WuU: 1.1399587e-06 
wyY: 2.283173e-27 
WaA: -0.00010309166 
WvV: -0.002395473005891688
The sum of each par -->
w1: -1.2701587e-06 
WuU: 1.1399587e-06 
wyY: 7.011115e-11 
WaA: -1.2156161e-05 
WvV: -0.0018982398508588848
The sum of each par -->
w1: -1.2701587e-06 
WuU: 1.1399587e-06 
wyY: 1.1879111e-20 
WaA: -0.00010380777 
WvV: -0.0022289032573945353
The sum of each par -->
w

alpha------------ [5.61935413e-02 5.04800443e-05 2.74955740e-06 1.76821119e-05
 1.03417113e-05 3.62077335e-05 5.45200746e-05 7.89680745e-06
 1.03409006e-05 2.83033713e-05]
softmax alpha-------------- [0.10517003 0.09942816 0.09942341 0.0994249  0.09942417 0.09942674
 0.09942856 0.09942392 0.09942417 0.09942595]
2 98
The sum of each par -->
w1: -3.6109893e-06 
WuU: -2.6839134e-06 
wyY: 4.6932556e-21 
WaA: -0.00013937222 
WvV: -0.0024052666303976153
The sum of each par -->
w1: -3.6109893e-06 
WuU: -2.6839134e-06 
wyY: 4.2710916e-16 
WaA: -0.00010060901 
WvV: -0.003100214240611105
The sum of each par -->
w1: -3.6109893e-06 
WuU: -2.6839134e-06 
wyY: 1.1358377e-18 
WaA: 6.805775e-05 
WvV: -0.0020306897279896677
The sum of each par -->
w1: -3.6109893e-06 
WuU: -2.6839134e-06 
wyY: 4.9743184e-26 
WaA: -9.7393764e-05 
WvV: -0.0028451526778574413
The sum of each par -->
w1: -3.6109893e-06 
WuU: -2.6839134e-06 
wyY: 6.232624e-11 
WaA: 0.00010047586 
WvV: -0.003144443492272691
alpha------------ 

alpha------------ [4.76728574e-02 5.89450652e-05 4.28617674e-05 2.69247276e-06
 5.17696903e-05 1.18991769e-05 8.07874667e-06 3.07247074e-05
 1.18775516e-05 1.84348453e-05]
softmax alpha-------------- [0.10437066 0.09951761 0.09951601 0.09951202 0.0995169  0.09951293
 0.09951255 0.09951481 0.09951293 0.09951358]
2 98
The sum of each par -->
w1: 6.7399174e-07 
WuU: -2.8737472e-06 
wyY: 1.3660408e-10 
WaA: -2.7900924e-05 
WvV: 0.001651333019296521
The sum of each par -->
w1: 6.7399174e-07 
WuU: -2.8737472e-06 
wyY: -1.4467555e-27 
WaA: 1.6551143e-05 
WvV: 0.001232302174301282
The sum of each par -->
w1: 6.7399174e-07 
WuU: -2.8737472e-06 
wyY: -2.3938168e-14 
WaA: -5.407713e-05 
WvV: 0.0011981571368158942
The sum of each par -->
w1: 6.7399174e-07 
WuU: -2.8737472e-06 
wyY: -1.2492442e-20 
WaA: 8.010455e-05 
WvV: 0.0018181482202807079
The sum of each par -->
w1: 6.7399174e-07 
WuU: -2.8737472e-06 
wyY: 8.072797e-19 
WaA: -4.7492107e-05 
WvV: 0.0015090756457726956
alpha------------ [0.01109

2 98
The sum of each par -->
w1: 2.8272468e-06 
WuU: -9.5223345e-07 
wyY: -1.3006874e-14 
WaA: -1.8396287e-05 
WvV: -0.0016413340488664633
The sum of each par -->
w1: 2.8272468e-06 
WuU: -9.5223345e-07 
wyY: -4.4060347e-28 
WaA: 3.103745e-05 
WvV: -0.001568424155459474
The sum of each par -->
w1: 2.8272468e-06 
WuU: -9.5223345e-07 
wyY: 2.718909e-18 
WaA: 0.00010630845 
WvV: -0.001868247927691121
The sum of each par -->
w1: 2.8272468e-06 
WuU: -9.5223345e-07 
wyY: 2.5075132e-21 
WaA: 4.4276294e-06 
WvV: -0.001533495488559947
The sum of each par -->
w1: 2.8272468e-06 
WuU: -9.5223345e-07 
wyY: -3.1426606e-12 
WaA: 0.00013315301 
WvV: -0.0007930216860163829
alpha------------ [0.00834245 0.00642511 0.00723345 0.00598828 0.00976009]
softmax alpha-------------- [0.20015839 0.19977499 0.19993654 0.19968774 0.20044235]
3 78
The sum of each par -->
w1: 7.999075e-06 
WuU: -3.9955034e-06 
wyY: -4.571193e-27 
WaA: 5.8674254e-05 
WvV: 0.003842765505899564
The sum of each par -->
w1: 7.999075e-06 


WuU: -1.1332798e-13 
wyY: -1.486576e-14 
WaA: 8.0796525e-08 
WvV: 3.3520824307792866e-05
alpha------------ [3.25520693e-06 1.90894420e-05 3.69703271e-05 6.63739591e-05
 4.62834662e-05 1.25456312e-05 5.95400981e-05 9.22480397e-06
 5.15854908e-02 1.37974313e-05]
softmax alpha-------------- [0.09947108 0.09947265 0.09947443 0.09947736 0.09947536 0.099472
 0.09947668 0.09947167 0.10473666 0.09947213]
2 98
The sum of each par -->
w1: -4.4804306e-06 
WuU: 4.540517e-06 
wyY: -1.4161482e-26 
WaA: -9.109406e-06 
WvV: -0.002497765526540069
The sum of each par -->
w1: -4.4804306e-06 
WuU: 4.540517e-06 
wyY: -1.486576e-14 
WaA: -5.075599e-07 
WvV: -0.0011452872905756803
The sum of each par -->
w1: -4.4804306e-06 
WuU: 4.540517e-06 
wyY: -3.431633e-19 
WaA: -7.970918e-05 
WvV: -0.001853594901085972
The sum of each par -->
w1: -4.4804306e-06 
WuU: 4.540517e-06 
wyY: 6.970152e-21 
WaA: -9.695123e-06 
WvV: -0.0019809274619687497
The sum of each par -->
w1: -4.4804306e-06 
WuU: 4.540517e-06 
wyY: 6.283

# Get latent factor and Each weight

In [70]:
U, Y, A, E, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, W1, Wu, Wy, Wa, Wv,Beta])

In [71]:
print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
print('W1 weight shape: ',A1.shape)
print('Wu weight shape:',Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:',Aa.shape)
print('Wv weight shape:',Av.shape)
print('Beta shape:',B.shape)
print('Embedding shape:',E.shape)

User latent shape:  (100, 128)
photo latent shape:  (88, 128)
Auxilary latent shape:  (88, 128)
W1 weight shape:  (100, 128)
Wu weight shape: (100, 128, 128)
Wy weight shape: (88, 128, 128)
Wa weight shape: (100, 128, 128)
Wv weight shape: (100, 128, 3374)
Beta shape: (100, 150)
Embedding shape: (150, 3374)


array([[ 1.28755188e-13, -1.59495830e-08,  1.67970065e-08, ...,
        -6.79700233e-14, -1.15951595e-08,  2.96662903e-11],
       [ 7.16980017e-11, -1.62418325e-11, -1.24057414e-11, ...,
         8.91374561e-12, -2.79513977e-12, -1.84613626e-11],
       [-8.25461731e-18,  1.03062558e-11,  9.12969830e-18, ...,
         1.62189845e-11,  4.00709840e-14, -1.09470229e-20],
       ...,
       [-1.16417351e-24,  8.37993319e-26,  1.97979825e-19, ...,
         6.30885467e-31, -7.52699314e-31,  4.53611797e-30],
       [ 6.38316104e-22,  1.14164738e-17, -5.85400740e-23, ...,
        -2.08511815e-21, -1.43180177e-18,  3.81385353e-18],
       [ 2.83457591e-18, -5.11021429e-18,  1.61269826e-12, ...,
         1.17598116e-15,  1.54162794e-12,  2.98273906e-11]], dtype=float32)

# Testing Part

In [87]:
result=np.zeros((test_amount,88))
RS=np.zeros((test_amount,88))
#test_idx --> Test 的 index

test_yes_id=[]
for s in range(test_amount):
    print(s,test_idx[s])

    yes=[]
    sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha=np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        #print(test_idx[s])
        #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))
        
        #Observe each part in attention , below par are all (128,1)
        testW1 = np.sum(np.multiply(A1[test_idx[s]],A1[test_idx[s]]))
        WuUu = np.sum(np.multiply(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T),np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)))
        WyYy = np.sum(np.multiply(np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T),np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)))
        WaAa = np.sum(np.multiply(np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T),np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T)))
        WvVy = np.sum(np.multiply(np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T),np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T)))
        print('w1:',testW1,'\nWuU:',WuUu,'\nwyY:',WyYy,'\nWaA:',WaAa,'\nWvV:',WvVy)
        
        alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))))*r
    mul=np.zeros((1,128))
    print('alpha------------',alpha)
    print('softmax alpha--------------',softmax(alpha))
    for i in range(len(sample)):
        mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
    new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    for k in range(88):
        result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
#print(RS[s])

0 50
w1: 2.4544266e-11 
WuU: 1.8040577e-23 
wyY: 1.5945054e-05 
WaA: 3.1537473e-21 
WvV: 2.2159012677821794e-18
w1: 2.4544266e-11 
WuU: 1.8040577e-23 
wyY: 6.469689e-05 
WaA: 3.5369395e-21 
WvV: 1.8931885856131214e-18
w1: 2.4544266e-11 
WuU: 1.8040577e-23 
wyY: 1.1103042e-05 
WaA: 3.3975095e-21 
WvV: 2.0774474670348896e-18
w1: 2.4544266e-11 
WuU: 1.8040577e-23 
wyY: 2.2845748e-09 
WaA: 2.3888677e-21 
WvV: 2.0544184028555723e-18
w1: 2.4544266e-11 
WuU: 1.8040577e-23 
wyY: 4.354328e-05 
WaA: 3.317824e-21 
WvV: 2.6350840712423046e-18
alpha------------ [ 4.12867649e-10  3.36386184e-09 -1.77629468e-10 -3.56692412e-12
 -2.37017963e-09]
softmax alpha-------------- [0.2 0.2 0.2 0.2 0.2]
1 93
w1: 8.7250996e-08 
WuU: 6.1143304e-19 
wyY: 3.348209e-06 
WaA: 2.0352104e-16 
WvV: 1.628519329311597e-14
w1: 8.7250996e-08 
WuU: 6.1143304e-19 
wyY: 8.020492e-07 
WaA: 1.5566705e-16 
WvV: 2.0017965589172682e-14
w1: 8.7250996e-08 
WuU: 6.1143304e-19 
wyY: 2.2845748e-09 
WaA: 1.909407e-16 
WvV: 1.54082295262

In [91]:
#取出test的資料
testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*20
target = np.zeros((test_amount,yt_test_amount))
#test_t 是true的
#test_f 是false的
        
for z in range(test_amount):
    user_id = test_idx[z]
    #positive target YouTuber list
    youtube_t = test_t[z] 
    #not target YouTuber list
    youtube_f = test_f[z]
    
    #前兩個放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
        target[z][i] = 1
    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]

In [92]:
target

array([[1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]])

In [93]:
testRS

array([[ 1.39257779e+00,  8.39538025e-01,  3.52044755e-01,
         2.77313271e-01, -3.74751932e-02,  9.60353042e-01,
        -5.85731542e-01, -1.26045744e-01, -4.11231293e-01,
        -6.45579668e-01, -5.93694823e-01, -7.26156835e-01,
        -7.86830295e-01, -4.79643838e-01, -7.76845073e-01,
        -6.14529357e-01, -3.40669372e-01, -5.13845116e-01,
        -3.75563335e-01, -2.96375799e-01],
       [ 2.09959446e-02,  6.38371409e-01,  4.18734727e-02,
         3.38774903e-01, -1.12913081e-01,  1.94055487e-01,
         3.01452675e-01,  4.63157034e-01,  1.13397773e-01,
         4.92353276e-01,  2.44654448e-01,  1.51711594e-01,
        -3.04710122e-01,  2.88842688e-01, -5.87249992e-01,
        -2.15522622e-01, -8.75586000e-02, -1.33918882e-01,
        -4.73470441e-02, -3.98187132e-01],
       [ 1.52895763e-02, -1.37369815e-01, -6.30140955e-02,
         3.81419991e-01,  1.91668400e+00, -3.10534843e-01,
        -4.12069332e-01,  1.23111263e+00, -5.52604790e-01,
        -4.29534017e-01, -3.8

In [94]:
sumtarget = 0
for i in range(len(target)):
    #print(np.sum(target[i]))
    sumtarget += np.sum(target[i])
print('num of positive data in testing:',sumtarget)
print('total testing data:',test_amount*yt_test_amount)

num of positive data in testing: 68.0
total testing data: 200


In [95]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [96]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),int(np.sum(target[i])))
    count_0_all.append(top_0)
    print(top_0)

acc_0 = 0
total = 0
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        #print(int(np.sum(target[i])))
        total+=int(np.sum(target[i]))
        if count_0_all[i][j] < int(np.sum(target[i])): #代表是0或1 (也就是target)
            acc_0 += 1
avg_acc = acc_0/100
print('avg_accuarcy for count_0:',avg_acc)
print(acc_0)

[0, 5, 1, 2, 3, 4]
[1, 9, 7, 3, 6, 13, 10, 5, 11, 8, 2]
[4, 12, 7, 3, 16, 0, 2, 1]
[2, 19, 5, 16, 6, 4, 12]
[2, 0, 1, 5, 16, 6]
[12, 13, 6]
[4, 1, 5, 9, 8, 15, 10, 7, 0, 3]
[4, 0, 18, 1, 10]
[0, 3, 8, 18, 10, 9]
[15, 19, 17, 0, 14, 16]
avg_accuarcy for count_0: 0.43
43


In [97]:
acc_0

43

In [98]:
total

512

# Top 1 

In [99]:
def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [100]:
correct = 0
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),1) #取一個
    count_0_all.append(top_0)
    print(top_0)
    if top_0[0] < int(np.sum(target[i])):
        correct += 1

[0]
[1]
[4]
[2]
[2]
[12]
[4]
[4]
[0]
[15]


In [101]:
top1_prec = correct/len(testRS)
top1_recall = correct/(sumtarget)
print('prec ',top1_prec,'recall ',top1_recall)

prec  0.8 recall  0.11764705882352941


In [102]:
#f1 score
print('F1_score:',F1_score(top1_prec,top1_recall))

F1_score: 0.20512820512820512


# Top 3

In [103]:
correct = 0
for i in range(len(testRS)):
    top_3 = topN(list(testRS[i]),3) #取一個
    count_0_all.append(top_3)
    #print(top_3)
    for j in range(len(top_3)):
        if top_3[j] < int(np.sum(target[i])):
            correct += 1

In [104]:
top3_prec = correct/(len(testRS)*3)
top3_recall = correct/(sumtarget)
print('prec ',top3_prec,'recall ',top3_recall)

prec  0.6666666666666666 recall  0.29411764705882354


In [105]:
#f1 score
print('F1_score:',F1_score(top3_prec,top3_recall))

F1_score: 0.40816326530612246


# Top 5

In [106]:
correct = 0
for i in range(len(testRS)):
    top_5 = topN(list(testRS[i]),5) #取一個
    count_0_all.append(top_5)
    #print(top_5)
    for j in range(len(top_5)):
        if top_5[j] < int(np.sum(target[i])):
            correct += 1

In [107]:
top5_prec = correct/(len(testRS)*5)
top5_recall = correct/(sumtarget)
print('prec ',top5_prec,'recall ',top5_recall)

prec  0.64 recall  0.47058823529411764


In [108]:
#f1 score
print('F1_score:',F1_score(top5_prec,top5_recall))

F1_score: 0.5423728813559322


In [99]:
#User_latent_factor = loaddata['User']
#YouTuber_latent_factor = loaddata['YouTuber']

In [ ]:
np.savez('../Data/latent_factor/YRM_up10_ALL/Final/1226.npz', User=U, YouTuber=Y)

In [ ]:
def softmax(x):
    x = x - np.max(x)
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

In [ ]:
softmax([-0.000000000000000000000000000000000000001,0.00000000000000000000000000000000000001])